<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/multi_document_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 多文档代理

在本指南中，您将学习如何设置一个能够有效回答不同类型问题的代理，涉及更大范围的文档。

这些问题包括：

- 针对特定文档的问答
- 比较不同文档的问答
- 针对特定文档的摘要
- 比较不同文档的摘要

我们将使用以下架构实现这一目标：

- 在每个文档上设置一个“文档代理”：每个文档代理可以在其文档内进行问答/摘要
- 在这组文档代理上设置一个顶层代理。进行工具检索，然后对工具集进行协同训练以回答问题。


## 设置和下载数据

在这一部分，我们将定义导入内容，然后下载关于不同城市的维基百科文章。每篇文章都被单独存储。

我们加载了18个城市的文章 - 虽然还没有达到“数百”篇文档的级别，但已经足够大，需要进行一些顶层文档检索！


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

In [ ]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

In [ ]:
from pathlib import Pathimport requestsfor title in wiki_titles:    response = requests.get(        "https://en.wikipedia.org/w/api.php",        params={            "action": "query",            "format": "json",            "titles": title,            "prop": "extracts",            # 'exintro': True,            "explaintext": True,        },    ).json()    page = next(iter(response["query"]["pages"].values()))    wiki_text = page["extract"]    data_path = Path("data")    if not data_path.exists():        Path.mkdir(data_path)    with open(data_path / f"{title}.txt", "w") as fp:        fp.write(wiki_text)

In [ ]:
# 加载所有维基文档city_docs = {}for wiki_title in wiki_titles:    city_docs[wiki_title] = SimpleDirectoryReader(        input_files=[f"data/{wiki_title}.txt"]    ).load_data()

定义全局LLM和嵌入


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## 构建多文档代理

在本节中，我们将向您展示如何构建多文档代理。我们首先为每个文档构建一个文档代理，然后使用对象索引定义顶层父代理。


### 为每个文档构建文档代理

在本节中，我们为每个文档定义"文档代理"。

我们为每个文档定义了一个向量索引（用于语义搜索）和摘要索引（用于摘要生成）。然后，这两个查询引擎被转换为工具，传递给一个调用代理的OpenAI函数。

这个文档代理可以动态选择在给定文档中执行语义搜索或摘要生成。

我们为每个城市创建一个单独的文档代理。


In [ ]:
from llama_index.agent.openai import OpenAIAgentfrom llama_index.core import load_index_from_storage, StorageContextfrom llama_index.core.node_parser import SentenceSplitterimport osnode_parser = SentenceSplitter()# 构建代理字典agents = {}query_engines = {}# 这是为了基准线all_nodes = []for idx, wiki_title in enumerate(wiki_titles):    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])    all_nodes.extend(nodes)    if not os.path.exists(f"./data/{wiki_title}"):        # 构建向量索引        vector_index = VectorStoreIndex(nodes)        vector_index.storage_context.persist(            persist_dir=f"./data/{wiki_title}"        )    else:        vector_index = load_index_from_storage(            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),        )    # 构建摘要索引    summary_index = SummaryIndex(nodes)    # 定义查询引擎    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)    # 定义工具    query_engine_tools = [        QueryEngineTool(            query_engine=vector_query_engine,            metadata=ToolMetadata(                name="vector_tool",                description=(                    "用于回答与特定方面相关的问题"                    f" {wiki_title} (例如历史、艺术和文化、"                    "体育、人口统计等)。"                ),            ),        ),        QueryEngineTool(            query_engine=summary_query_engine,            metadata=ToolMetadata(                name="summary_tool",                description=(                    "用于需要全面摘要的任何请求"                    f" {wiki_title}。对于更具体部分的问题，请使用vector_tool。"                ),            ),        ),    ]    # 构建代理    function_llm = OpenAI(model="gpt-4")    agent = OpenAIAgent.from_tools(        query_engine_tools,        llm=function_llm,        verbose=True,        system_prompt=f"""\您是一个专门设计用于回答关于 {wiki_title} 的查询的代理。在回答问题时，您必须始终使用提供的工具之一；不要依赖先前的知识。\""",    )    agents[wiki_title] = agent    query_engines[wiki_title] = vector_index.as_query_engine(        similarity_top_k=2    )

### 构建Retriever-Enabled OpenAI Agent

我们构建一个顶层代理，可以协调不同的文档代理来回答任何用户查询。

这个代理接受所有文档代理作为工具。这个特定的代理`RetrieverOpenAIAgent`在使用工具之前执行工具检索（与默认代理尝试将所有工具放在提示中的方式不同）。

在这里，我们使用了一个top-k检索器，但我们鼓励您自定义工具检索方法！


In [ ]:
# 为每个文档代理定义工具all_tools = []for wiki_title in wiki_titles:    wiki_summary = (        f"这个内容包含了关于{wiki_title}的维基百科文章。如果你想回答任何关于{wiki_title}的问题，可以使用这个工具。\n"    )    doc_tool = QueryEngineTool(        query_engine=agents[wiki_title],        metadata=ToolMetadata(            name=f"tool_{wiki_title}",            description=wiki_summary,        ),    )    all_tools.append(doc_tool)

In [ ]:
# 定义一个在这些工具上进行索引和检索的“对象”索引器from llama_index.core import VectorStoreIndexfrom llama_index.core.objects import ObjectIndexobj_index = ObjectIndex.from_objects(    all_tools,    index_cls=VectorStoreIndex,)

In [ ]:
来自llama_index.agent.openai的OpenAIAgenttop_agent = OpenAIAgent.from_tools(    tool_retriever=obj_index.as_retriever(similarity_top_k=3),    system_prompt=""" \您是一个专门回答关于一组给定城市的查询的代理。请始终使用提供的工具来回答问题。不要依赖先前的知识。\""",    verbose=True,)

### 定义基准向量存储索引

作为比较的基准，我们定义了一个“朴素”的RAG管道，将所有文档都存储在单个向量索引集合中。

我们设置top_k = 4


In [ ]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

## 运行示例查询

让我们运行一些示例查询，涵盖从针对单个文档的问答/摘要到针对多个文档的问答/摘要。


In [ ]:
# 应该使用波士顿代理 -> 矢量工具response = top_agent.query("告诉我有关波士顿的艺术和文化")

=== Calling Function ===
Calling function: tool_Boston with args: {
  "input": "arts and culture"
}
=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "arts and culture"
}
Got output: Boston is known for its vibrant arts and culture scene. The city is home to a number of performing arts organizations, including the Boston Ballet, Boston Lyric Opera Company, Opera Boston, Boston Baroque, and the Handel and Haydn Society. There are also several theaters in or near the Theater District, such as the Cutler Majestic Theatre, Citi Performing Arts Center, the Colonial Theater, and the Orpheum Theatre. Boston is a center for contemporary classical music, with groups like the Boston Modern Orchestra Project and Boston Musica Viva. The city also hosts major annual events, such as First Night, the Boston Early Music Festival, and the Boston Arts Festival. In addition, Boston has several art museums and galleries, including the Museum of Fine Arts, the Isabella Stewart 

In [ ]:
print(response)

Boston has a rich arts and culture scene, with a variety of performing arts organizations and venues. The city is home to renowned institutions such as the Boston Ballet, Boston Lyric Opera Company, Opera Boston, Boston Baroque, and the Handel and Haydn Society. The Theater District in Boston is a hub for theatrical performances, with theaters like the Cutler Majestic Theatre, Citi Performing Arts Center, Colonial Theater, and Orpheum Theatre.

In addition to performing arts, Boston also has a thriving contemporary classical music scene, with groups like the Boston Modern Orchestra Project and Boston Musica Viva. The city hosts several annual events that celebrate the arts, including First Night, the Boston Early Music Festival, and the Boston Arts Festival.

Boston is also known for its visual arts scene, with a number of art museums and galleries. The Museum of Fine Arts, the Isabella Stewart Gardner Museum, and the Institute of Contemporary Art are among the notable institutions in 

In [ ]:
# 基线response = base_query_engine.query(    "告诉我有关波士顿的艺术和文化")print(str(response))

Boston has a rich arts and culture scene. The city is home to a variety of performing arts organizations, such as the Boston Ballet, Boston Lyric Opera Company, Opera Boston, Boston Baroque, and the Handel and Haydn Society. Additionally, there are numerous contemporary classical music groups associated with the city's conservatories and universities, like the Boston Modern Orchestra Project and Boston Musica Viva. The Theater District in Boston is a hub for theater, with notable venues including the Cutler Majestic Theatre, Citi Performing Arts Center, the Colonial Theater, and the Orpheum Theatre. Boston also hosts several significant annual events, including First Night, the Boston Early Music Festival, the Boston Arts Festival, and the Boston gay pride parade and festival. The city is renowned for its historic sites connected to the American Revolution, as well as its art museums and galleries, such as the Museum of Fine Arts, Isabella Stewart Gardner Museum, and the Institute of C

In [ ]:
# 应该使用Houston代理 -> 向量工具response = top_agent.query(    "给我一个关于Houston所有积极方面的总结")

=== Calling Function ===
Calling function: tool_Houston with args: {
  "input": "positive aspects"
}
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "positive aspects"
}
Got output: Houston has many positive aspects that make it an attractive place to live and visit. The city's diverse population, with people from different ethnic and religious backgrounds, adds to its cultural richness and inclusiveness. Additionally, Houston is home to the Texas Medical Center, which is the largest concentration of healthcare and research institutions in the world. The presence of NASA's Johnson Space Center also highlights Houston's importance in the fields of medicine and space exploration. The city's strong economy, supported by industries such as energy, manufacturing, aeronautics, and transportation, provides numerous economic opportunities for residents and visitors alike. Furthermore, Houston has a thriving visual and performing arts scene, including a theater d

In [ ]:
print(response)

Houston has numerous positive aspects that make it a desirable place to live and visit. Some of these include:

1. Diversity: Houston is known for its diverse population, with people from different ethnic and religious backgrounds. This diversity adds to the city's cultural richness and inclusiveness.

2. Healthcare and Research Institutions: The city is home to the Texas Medical Center, the largest concentration of healthcare and research institutions in the world. This makes Houston a hub for medical innovation and healthcare services.

3. Space Exploration: Houston is also known for NASA's Johnson Space Center, highlighting the city's significant role in space exploration.

4. Strong Economy: Houston's economy is robust and diverse, supported by industries such as energy, manufacturing, aeronautics, and transportation. This provides numerous economic opportunities for its residents.

5. Arts and Culture: The city has a thriving visual and performing arts scene, with a theater distri

In [ ]:
# 基线response = base_query_engine.query(    "给我一个关于休斯顿所有积极方面的总结")print(str(response))

Houston has several positive aspects that contribute to its reputation as a thriving city. It is home to a diverse and growing international community, with a large number of foreign banks and consular offices representing 92 countries. The city has received numerous accolades, including being ranked as one of the best cities for employment, college graduates, and homebuyers. Houston has a strong economy, with a broad industrial base in sectors such as energy, manufacturing, aeronautics, and healthcare. It is also a major center for the oil and gas industry and has the second-most Fortune 500 headquarters in the United States. The city's cultural scene is vibrant, with a variety of annual events celebrating different cultures, as well as a reputation for diverse and excellent food. Houston is known for its world-class museums and performing arts scene. Additionally, the city has made significant investments in renewable energy sources like wind and solar. Overall, Houston offers a high

In [ ]:
＃基线：响应与来源不完全匹配...response.source_nodes[1].get_content()

In [ ]:
response = top_agent.query(
    "Tell the demographics of Houston, and then compare that with the"
    " demographics of Chicago"
)

=== Calling Function ===
Calling function: tool_Houston with args: {
  "input": "demographics"
}
=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "demographics"
}
Got output: Houston is a majority-minority city with a diverse population. According to the U.S. Census Bureau, in 2019, non-Hispanic whites made up 23.3% of the population, Hispanics and Latino Americans 45.8%, Blacks or African Americans 22.4%, and Asian Americans 6.5%. The largest Hispanic or Latino American ethnic group in the city is Mexican Americans, followed by Puerto Ricans and Cuban Americans. Houston is also home to the largest African American community west of the Mississippi River. Additionally, Houston has a growing Muslim population, with Muslims estimated to make up 1.2% of the city's population. The city is known for its LGBT community and is home to one of the largest pride parades in the United States. The Hindu, Sikh, and Buddhist communities are also growing in Houston. Over

In [ ]:
print(response)

Houston has a diverse population with a demographic makeup that includes non-Hispanic whites (23.3%), Hispanics and Latino Americans (45.8%), Blacks or African Americans (22.4%), and Asian Americans (6.5%). The largest Hispanic or Latino American ethnic group in Houston is Mexican Americans. Houston is also home to the largest African American community west of the Mississippi River and has a growing Muslim population.

On the other hand, Chicago is also known for its diverse demographics. The city has a significant non-Hispanic White population, along with a substantial Black population and Hispanic population. Chicago is celebrated for its cultural diversity and has a significant LGBT population.

Both Houston and Chicago have diverse populations, with a mix of different racial and ethnic groups contributing to their vibrant communities.


In [ ]:
# 基线response = base_query_engine.query(    "告诉我休斯顿的人口统计信息，然后将其与芝加哥的人口统计信息进行比较")print(str(response))

Houston is the most populous city in Texas and the fourth-most populous city in the United States. It has a population of 2,304,580 as of the 2020 U.S. census. The city is known for its diversity, with a significant proportion of minorities. In 2019, non-Hispanic whites made up 23.3% of the population, Hispanics and Latino Americans 45.8%, Blacks or African Americans 22.4%, and Asian Americans 6.5%. The largest Hispanic or Latino American ethnic group in Houston is Mexican Americans, comprising 31.6% of the population.

In comparison, Chicago is the third-most populous city in the United States. According to the 2020 U.S. census, Chicago has a population of 2,746,388. The demographics of Chicago are different from Houston, with non-Hispanic whites making up 32.7% of the population, Hispanics and Latino Americans 29.9%, Blacks or African Americans 29.8%, and Asian Americans 7.6%. The largest Hispanic or Latino American ethnic group in Chicago is Mexican Americans, comprising 21.6% of th

In [ ]:
# 基准线：该响应对芝加哥没有提供任何信息...response.source_nodes[3].get_content()

In [ ]:
response = top_agent.query(
    "Tell me the differences between Shanghai and Beijing in terms of history"
    " and current economy"
)

=== Calling Function ===
Calling function: tool_Shanghai with args: {
  "input": "history"
}
=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "history"
}
Got output: Shanghai has a rich history that dates back to ancient times. However, in the context provided, the history of Shanghai is mainly discussed in relation to its modern development. After the war, Shanghai's economy experienced significant growth, with increased agricultural and industrial output. The city's administrative divisions were rearranged, and it became a center for radical leftism during the 1950s and 1960s. The Cultural Revolution had a severe impact on Shanghai's society, but the city maintained economic production with a positive growth rate. Shanghai also played a significant role in China's Third Front campaign and has been a major contributor of tax revenue to the central government. Economic reforms were initiated in Shanghai in 1990, leading to the development of the Pudong dis

In [ ]:
print(str(response))

In terms of history, both Shanghai and Beijing have rich and complex pasts. Shanghai's history dates back to ancient times, but its modern development is particularly noteworthy. It experienced significant economic growth after the war and played a major role in China's economic reforms. Beijing, on the other hand, has a history that spans several dynasties and served as the capital during the Ming and Qing dynasties. It has preserved its historical heritage while evolving into a modern metropolis.

In terms of current economy, Shanghai is a global center for finance and innovation. It has a diverse economy and has experienced rapid development, with a high GDP and significant foreign investment. It is a major player in the global financial industry and is home to the Shanghai Stock Exchange. Beijing's economy is primarily driven by the tertiary sector, with a focus on services such as professional services, information technology, and commercial real estate. It has identified high-end

In [ ]:
# 基线response = base_query_engine.query(    "告诉我上海和北京在历史和当前经济方面的区别")print(str(response))

Shanghai and Beijing have distinct differences in terms of history and current economy. Historically, Shanghai was the largest and most prosperous city in East Asia during the 1930s, while Beijing served as the capital of the Republic of China and later the People's Republic of China. Shanghai experienced significant growth and redevelopment in the 1990s, while Beijing expanded its urban area and underwent rapid development in the last two decades.

In terms of the current economy, Shanghai is considered the "showpiece" of China's booming economy. It is a global center for finance and innovation, with a strong focus on industries such as retail, finance, IT, real estate, machine manufacturing, and automotive manufacturing. Shanghai is also home to the world's busiest container port, the Port of Shanghai. The city has a high GDP and is classified as an Alpha+ city by the Globalization and World Cities Research Network.

On the other hand, Beijing is a global financial center and ranks t